In [109]:


import TetrisUtils
from test_TetrisUtils import board1,board2,board3

In [110]:
def speed_aggregate_height():
    for i in range(10000):
        height = TetrisUtils.get_aggregate_height(board1)
        height = TetrisUtils.get_aggregate_height(board2)
        height = TetrisUtils.get_aggregate_height(board3)

def speed_hollow_column_count():
    for i in range(10000):
        height = TetrisUtils.get_hollow_column_count(board1)
        height = TetrisUtils.get_hollow_column_count(board2)
        height = TetrisUtils.get_hollow_column_count(board3)
def speed_row_transition():
    for i in range(10000):
        height = TetrisUtils.get_row_transition(board1)
        height = TetrisUtils.get_row_transition(board2)
        height = TetrisUtils.get_row_transition(board3)
def speed_col_transition():
    for i in range(10000):
        height = TetrisUtils.get_col_transition(board1)
        height = TetrisUtils.get_col_transition(board2)
        height = TetrisUtils.get_col_transition(board3)
def speed_get_pit_count():
    for i in range(10000):
        height = TetrisUtils.get_pit_count(board1)
        height = TetrisUtils.get_pit_count(board2)
        height = TetrisUtils.get_pit_count(board3)

def speed_get_bumpiness():
    for i in range(10000):
        height = TetrisUtils.get_bumpiness(board1)
        height = TetrisUtils.get_bumpiness(board2)
        height = TetrisUtils.get_bumpiness(board3)
def speed_get_board_and_lines_cleared():
    for i in range(10000):
        height = TetrisUtils.get_board_and_lines_cleared(board1)
        height = TetrisUtils.get_board_and_lines_cleared(board2)
        height = TetrisUtils.get_board_and_lines_cleared(board3)
print(TetrisUtils.get_aggregate_height(board1))
print(TetrisUtils.get_aggregate_height(board2))
print(TetrisUtils.get_aggregate_height(board3))

10
20
24


In [111]:
%time speed_aggregate_height()
%time  speed_hollow_column_count()
%time speed_row_transition()
%time speed_col_transition()
%time speed_get_pit_count()
%time speed_get_bumpiness()
%time speed_get_board_and_lines_cleared()

CPU times: total: 172 ms
Wall time: 179 ms
CPU times: total: 156 ms
Wall time: 149 ms
CPU times: total: 109 ms
Wall time: 115 ms
CPU times: total: 203 ms
Wall time: 211 ms
CPU times: total: 125 ms
Wall time: 115 ms
CPU times: total: 188 ms
Wall time: 187 ms
CPU times: total: 15.6 ms
Wall time: 17 ms


# Numpy version

In [112]:
import numpy as np

In [113]:
board1 = np.array(board1)
board2 = np.array(board2)
board3 = np.array(board3)
board1



array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]])

In [114]:
board1.nonzero()
boolean_board = board1.astype(bool)

height = board1.shape[0] - np.argmax(boolean_board,axis=0)
#board1.shape[0]
height

height[height==board1.shape[0]] = 0
height
np.sum(height)
#height = np.where(height!=board1.shape[0])
#height
#sum(np.count_nonzero(board1,axis=0))

10

In [115]:
board1[5,2]

1

In [116]:
def aggregate_height(board):
    board_len = board.shape[0]
    boolean_board = board.astype(bool)
    height = board_len - np.argmax(boolean_board,axis=0)
    height[height==board_len] = 0
    return np.sum(height)

print(aggregate_height(board1))
print(aggregate_height(board2))
print(aggregate_height(board3))

10
20
24


In [117]:
def numpy_aggregate_height():
    for i in range(10000):
        height = aggregate_height(board1)
        height = aggregate_height(board2)
        height = aggregate_height(board3)

In [118]:
%time numpy_aggregate_height()

CPU times: total: 266 ms
Wall time: 260 ms


# Speedy version

In [119]:
from numba import jit,prange
from numba.typed import List

In [120]:
numba_board1 = List(board1)
numba_board2 = List(board2)
numba_board3 = List(board3)

In [121]:
@jit(nopython=True)
def get_aggregate_height(board: List) -> int:
    GRID_COL_COUNT = len(board[0])
    GRID_ROW_COUNT = len(board)
    aggregate_height = 0
    column_checked = List()
    for row in range(GRID_ROW_COUNT):
        for col in range(GRID_COL_COUNT):
            if not (col in column_checked) and board[row][col] != 0:
                height = len(board) - row
                aggregate_height += height
                column_checked.append(col)

    return aggregate_height


height = get_aggregate_height(numba_board1)

In [122]:
def loop_speedy():
    for i in range(10000):
        height = get_aggregate_height(numba_board1)
        height = get_aggregate_height(numba_board2)
        height = get_aggregate_height(numba_board3)

In [123]:
%time loop_speedy()

CPU times: total: 62.5 ms
Wall time: 68.9 ms


## Col transition

In [124]:
@jit(nopython=True)
def get_col_transition(board):
    GRID_COL_COUNT = len(board[0])
    GRID_ROW_COUNT = len(board)

    nb_col_transition = 0
    for col in range(GRID_COL_COUNT):
        is_empty = True if board[0][col] == 0 else False
        for row in range(GRID_ROW_COUNT):
            if board[row][col] == 0 and not is_empty:
                nb_col_transition += 1
                is_empty = True
            elif board[row][col] != 0 and is_empty:
                nb_col_transition += 1
                is_empty = False
    return nb_col_transition


height = get_col_transition(numba_board1)

In [125]:
def loop_speedy_col_trans():
    for i in range(10000):
        height = get_col_transition(numba_board1)
        height = get_col_transition(numba_board2)
        height = get_col_transition(numba_board3)

In [126]:
%time loop_speedy_col_trans()

CPU times: total: 93.8 ms
Wall time: 75.1 ms


### Parralel

In [133]:
@jit(nopython=True)
def get_col_transition_parallel(board):
    GRID_COL_COUNT = len(board[0])
    GRID_ROW_COUNT = len(board)

    nb_col_transition = 0
    col_transitions = np.zeros(GRID_ROW_COUNT)

    for row in range(GRID_ROW_COUNT):
        for col in prange(GRID_COL_COUNT):
            col_transition = 0
            if (board[row][col] == 0 and board[row][col+1] !=0) or (board[row][col] != 0 and board[row][col+1] ==0) :
                col_transitions[row]+=1

    return col_transitions.sum()

height = get_col_transition_parallel(numba_board1)

In [134]:
def loop_speedy_col_trans_par():
    for i in range(10000):
        height = get_col_transition_parallel(numba_board1)
        height = get_col_transition_parallel(numba_board2)
        height = get_col_transition_parallel(numba_board3)

In [135]:
%time loop_speedy_col_trans_par()

CPU times: total: 62.5 ms
Wall time: 60.1 ms
